In [ ]:
import csv
import numpy as np
import jwt
import urllib.request
import os
from time import sleep

In [ ]:
with open("attachments_amount_jwt.csv") as f:
    reader = csv.DictReader(f)
    attachments = list(reader)

In [ ]:
for i in range(len(attachments)):
    attachments[i]['filename'] = (
        str(jwt.decode(attachments[i]['jwt'], verify=False)['account'])
        + '-'
        + jwt.decode(attachments[i]['jwt'], verify=False)['filename']
    ).strip()

In [ ]:
for i in range(len(attachments)):
    # decode jwt to create file name by concatenating jwt account and jwt filename
    attachments[i]['filename'] = (
        str(jwt.decode(attachments[i]['jwt'], verify=False)['account'])
        + '-'
        + jwt.decode(attachments[i]['jwt'], verify=False)['filename']
    ).strip()
    
    # identify file type
    with urllib.request.urlopen("https://api.freshbooks.com/uploads/attachments/" + attachments[i]['jwt']) as response:
        info = response.info()
        attachments[i]['filetype'] = info.get_content_subtype()
    
    # save file with file name and file type, waiting 1 second between each urlretrieve request
    if info.get_content_subtype() == 'jpeg':
        urllib.request.urlretrieve("https://api.freshbooks.com/uploads/attachments/" + attachments[i]['jwt'],
                                   os.path.join('images', attachments[i]['filename'] + '.jpg'))
        sleep(1)
    elif  info.get_content_subtype() == 'png':
        urllib.request.urlretrieve("https://api.freshbooks.com/uploads/attachments/" + attachments[i]['jwt'],
                                   os.path.join('images', attachments[i]['filename'] + '.png'))
        sleep(1)
#     else:
#         print('Error at: ', i, ' Corresponding to jwt: ', attachments[i]['jwt'], ' With file type: ', info.get_content_subtype())
#         break

In [ ]:
csv_columns = ['amount','jwt','filename','filetype']
with open('attachments_amount_jwt_output.csv', mode='w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=csv_columns)
    writer.writeheader()
    for data in attachments:
        writer.writerow(data)

In [ ]:
attachments